In [1]:
import os


In [ ]:
# os.chdir("../")
%pwd

'd:\\Projects\\Machine Learning\\MLOPS'

In [34]:
# pip install dagshub

In [4]:
import dagshub

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/ChaudhariJayant/MLOPS.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "ChaudhariJayant"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "7cd6389315db46e69eb4b961f3de03abe3b0dde8"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path:Path
    model_path:Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri:str

In [7]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config =self.config.model_evaluation
        params =self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path= config.model_path,
            all_params=params,
            metric_file_name= config.metric_file_name,
            target_column=schema.name,
            mlflow_uri= "https://dagshub.com/ChaudhariJayant/MLOPS.mlflow"
        )
        return model_evaluation_config

In [9]:
import os 
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [17]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rsme = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rsme, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop(self.config.target_column, axis=1)
        test_y = test_data[[self.config.target_column]]

        # mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        # logger.info(f"url fuck up {urlparse(mlflow.get_tracking_uri())}")
        

        with mlflow.start_run():
            
            predicted_qualities = model.predict(test_x)
            # print(predicted_qualities)

            (rsme, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            
            scores = {"rsme":rsme, "mae":mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            
            
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rsmr", rsme)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
                       

            # mlflow.sklearn.log_model(sk_model= model, name="ElasticnetModel")
            # This will log your model as a file without trying to "register" it

            # if tracking_url_type_store != "file":

            #     # Register the model
            #     # There are other ways to use the Model Registry, which depends on the use case,
            #     # please refer to the doc for more information:
            #     # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            #     mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            # else:
            #     mlflow.sklearn.log_model(model, "model")

    
            mlflow.log_artifact(local_path=self.config.model_path, artifact_path="ElasticnetModel")
            # mlflow.sklearn.log_model(sk_model=model)
           

New Class


In [18]:

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-11-16 15:42:40,690: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-16 15:42:40,692: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-16 15:42:40,694: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-16 15:42:40,696: INFO: common: created directory at: artifacts]
[2025-11-16 15:42:40,697: INFO: common: created directory at: artifacts/model_evalution]


c:\Users\chaud\anaconda3\envs\ML_venv\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.3.0 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[2025-11-16 15:42:40,975: INFO: common: json file saved at: artifacts\model_evalution\metrics.json]
🏃 View run serious-hog-194 at: https://dagshub.com/ChaudhariJayant/MLOPS.mlflow/#/experiments/0/runs/4e689e8754ba45629316c0309bee59f5
🧪 View experiment at: https://dagshub.com/ChaudhariJayant/MLOPS.mlflow/#/experiments/0
